In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time
import pandas as pd

In [71]:
adblock = "AdBlock.crx"
option = webdriver.EdgeOptions()
option.add_argument(adblock)
driver = webdriver.Edge(options=option)

In [120]:
url = "https://www.dolarito.ar/cotizaciones-historicas/informal"
driver.get(url)

In [126]:
def change_date(date, year_, month_, start = True):
    if start: 
        index = 0 
        driver.execute_script("arguments[0].scrollIntoView();", date)
    else: index = -1
    
    date.click()

    sel_mes = Select(driver.find_element(
        By.CLASS_NAME, "react-datepicker__month-select"))
    sel_ano = Select(driver.find_element(
        By.CLASS_NAME, "react-datepicker__year-select"))

    for month in sel_mes.options:
        if month_ in month.text:
            sel_mes.select_by_visible_text(month.text.lower())
            
    for year in sel_ano.options:
        if int(year.text) == year_:
            sel_ano.select_by_value(year.text)


    week = driver.find_elements(By.CLASS_NAME, "react-datepicker__week")
    week = week[index].find_elements(By.TAG_NAME, "div")
    days = [x for x in week if x.text]
    days[index].click()

ingresos = pd.read_csv("Datasets/Ingresos.csv")
periodo = ingresos["Periodo"].to_list()
ingresos = ingresos["Ingresos (miles de pesos)"].to_list()
periodo.reverse()
ingresos.reverse()

blue_column = []
for i ,fecha in enumerate(periodo):
    mes, ano = fecha.split(" ")
    mes = mes.split("-")
    
    date = driver.find_elements(By.CSS_SELECTOR, ".chakra-input.date-picker-input.css-f3ghr7")[0]
    change_date(date, int(ano), mes[0])
    
    date = driver.find_elements(By.CSS_SELECTOR, ".chakra-input.date-picker-input.css-f3ghr7")[1]
    change_date(date, int(ano), mes[1], start=False)
    blue = driver.find_elements(By.CSS_SELECTOR, ".chakra-stat__help-text.css-uymg32")
    # driver.execute_script("arguments[0].scrollIntoView();", blue)

    time.sleep(1)
    promedio = [x.text for x in blue if "Promedio" in x.text][-1]
    promedio = float(promedio.split("$")[-1].replace(",","."))
    
    dolar = ingresos[i] / promedio
    blue_column.append(dolar)
    
    date = driver.find_elements(By.CSS_SELECTOR, ".chakra-input.date-picker-input.css-f3ghr7")[1]
    change_date(date, int(ano) + 1, mes[1], start=False)




KeyboardInterrupt: 